In [1]:
from ref_symbolic_mod import *
# import ref_symbolic_mod_old as old 
import sympy as sym
# display = IPython.display.display
from itertools import chain, combinations
from collections import defaultdict

%load_ext autoreload
%autoreload 2

In [14]:
a, b = sym.symbols('a b')

A = Matrix([[a,2,a,a+b],
            [a,2,a,a],
            [3,3,-b,3],
            [a+1,3,a+1,a+1]])

B = Matrix([a-b, a-b, -b, a-b+1])
display([A, B])

⎡⎡  a    2    a    a + b⎤  ⎡  a - b  ⎤⎤
⎢⎢                      ⎥  ⎢         ⎥⎥
⎢⎢  a    2    a      a  ⎥  ⎢  a - b  ⎥⎥
⎢⎢                      ⎥, ⎢         ⎥⎥
⎢⎢  3    3   -b      3  ⎥  ⎢   -b    ⎥⎥
⎢⎢                      ⎥  ⎢         ⎥⎥
⎣⎣a + 1  3  a + 1  a + 1⎦  ⎣a - b + 1⎦⎦

In [20]:
A.aug_line(-5)
A

IndexError: Position for augmented line (-5) out of range (4).

In [4]:
isinstance(A, Matrix) and hasattr(A, 'aug_pos') and len(A.aug_pos) > 0

True

In [18]:
display(A)

<IPython.core.display.Math object>

In [24]:
C = sym.Matrix([[a,2,a,a+b],
            [a,2,a,a],
            [3,3,-b,3],
            [a+1,3,a+1,a+1]])
C.__repr__()

'Matrix([\n[    a, 2,     a, a + b],\n[    a, 2,     a,     a],\n[    3, 3,    -b,     3],\n[a + 1, 3, a + 1, a + 1]])'

In [5]:
display(A)

IndexError: list index out of range

In [10]:
for i in {1, 2, 3}:
    print(i)

1
2
3


In [17]:
C = A.aug_line().row_join(B)
C.augpos

{4, 5}

In [31]:
display(IPython.display.Math('$$\\left[\\begin{array}{cccc|c}a & 2 & a & a + b & a - b\\\\a & 2 & a & a & a - b\\\\3 & 3 & - b & 3 & - b\\\\a + 1 & 3 & a + 1 & a + 1 & a - b + 1\\end{array}\\right]$$'))

<IPython.core.display.Math object>

In [ ]:
"\begin{pmatrix}[cc|c]  1 & 2 & 3 \\4 & 5 & 9 \end{pmatrix}"

In [9]:
def cprint(m, augpos):
    # raw = sym.latex(m).replace("matrix", "array")
    raw = sym.latex(m, mat_str="array")
    print(raw)
    # create formatting string s to insert augment line visually
    ls = [pos for pos in augpos if 0 < pos < m.cols]
    ls.sort()
    delta = [ls[0]]
    delta.extend([ls[i] - ls[i-1] for i in range(1, len(ls))])
    remainder = m.cols - sum(delta)
    delta.append(remainder)
    # s = '{' + '|'.join(['c'*i for i in delta]) + '}'

    default_str = 'c'*m.cols
    formatted = raw.replace('c'*m.cols,  '|'.join(['c'*i for i in delta]))
    display(IPython.display.Math(formatted))
    # display(formatted)

cprint(C,C.aug_pos)

\left[\begin{array}{ccccc}a & 2 & a & a + b & a - b\\a & 2 & a & a & a - b\\3 & 3 & - b & 3 & - b\\a + 1 & 3 & a + 1 & a + 1 & a - b + 1\end{array}\right]


<IPython.core.display.Math object>

In [47]:
augpos = [1, 4, 5, 6, 3]
delta = [augpos[0]]
delta.extend([augpos[i] - augpos[i-1] for i in range(1, len(augpos))])
s = '|'.join(['c'*i for i in delta])
s

'c|ccc|c|c|'

In [13]:
[0, 1, 2, 3][-4]

0

In [ ]:
def remap(ls, pos):
    if pos < 0:
        pos += len(ls)

In [48]:
augpos.sort()
augpos

[1, 3, 4, 5, 6]

In [7]:
C = A.aug_line().row_join(B).aug_line().aug_line()


In [8]:
from typing import Callable
def ref(self, 
        verbosity: int = 2,
        max_tries: int = 2,
        follow_GE: bool = False,
        iszerofunc: Callable = is_zero,
        matrices: int = 2) -> sym.Matrix:
    # follow_GE can be set to True to follow Gaussian Elimination strictly
    # LU decomposition
    # matrices is the number of matrices return.
    # 1. Upper
    # 2. Perm @ Lower Upper
    # 3. Perm Lower Upper
    U = self.copy()

    I = self.id()
    L = self.id()
    P = self.id()

    # Loop over each column
    cur_row_pos = 0
    for cur_col_pos in range(self.cols):
        # Find the first non-zero row in the current column
        # print(cur_row_pos, cur_col_pos)
        pivot_row = U.get_pivot_row(cur_col_pos, cur_row_pos, follow_GE)
        # print(f"{pivot_row=}")

        if pivot_row == -1:
            # If no non-zero pivot is found, continue to the next column
            continue

        # Swap the current row with the pivot row if necessary
        if pivot_row != cur_row_pos:
            U.swap_row(cur_row_pos, pivot_row, verbosity=verbosity)
            P_elem = I.copy().swap_row(cur_row_pos, pivot_row)
            P = P @ P_elem
            L = P_elem @ L @ P_elem

        # Eliminate the current column in rest of the rows below
        for row_idx in range(cur_row_pos+1, self.rows):
            # reduce the row_idx iteratively via partial fractions to
            # prevent division by a possible 0 term
            tries = 0
            while U[row_idx, cur_col_pos] != 0:
                tries += 1
                if tries > max_tries:
                    print(f"ERROR: Max tries exceeded to reduce row {row_idx+1} with row {cur_row_pos+1}")
                    break
                try:
                    scalar = U[row_idx, cur_col_pos] / U[cur_row_pos, cur_col_pos] 
                    scalar = scalar.expand().simplify()
                    # print(f'{scalar=}')
                    try:
                        decomp = sym.apart(scalar) # partial fractions
                    except:
                        decomp = scalar
                    # if isinstance(decomp, sym.core.add.Add) and (scalar != decomp):
                    if isinstance(decomp, sym.core.add.Add):
                        terms = decomp.args
                    else:
                        # there is only 1 term (could be integer or proper fraction)
                        terms = [decomp]
                    # print(f'{terms=}')
                    for term in terms:
                        _, d = sym.fraction(term)
                        # print(f'{d=}')
                        # ensure denominator is non-zero so that reduction is valid

                        if (not iszerofunc(d)):
                            U.reduce_row(row_idx, term, cur_row_pos, verbosity=verbosity)
                            elem = I.copy().reduce_row(row_idx, -term, cur_row_pos)
                            L = L @ elem

                    # Cases where pivot row contains symbols such that scalar is a
                    # fraction with symbolic denominator.
                    # To reduce further, can only scale row_idx accordingly
                    if U[row_idx, cur_col_pos] != 0:
                        scalar = U[cur_row_pos, cur_col_pos] / U[row_idx, cur_col_pos]
                        scalar.simplify()
                        n, d = sym.fraction(scalar)
                        # to scale by n, n cannot be 0 both numerically or symbolically
                        # to scale by 1/d, d cannot be 0, same argument as n
                        # if (n != 0) and (not is_zero(d)):
                        if (not iszerofunc(n)) and (not iszerofunc(d)):
                            U.scale_row(row_idx, scalar, verbosity=verbosity)
                            elem = I.copy().scale_row(row_idx, 1/scalar)
                            L = L @ elem

                except Exception as error:
                    print(f"Exception encountered: {error}")
                    if matrices == 1:
                        return U
                    elif matrices == 2:
                        return P @ L, U
                    elif matrices == 3:
                        return P, L, U
                    else:
                        return U

        cur_row_pos += 1

    if matrices == 1:
        return U
    elif matrices == 2:
        return P @ L, U
    elif matrices == 3:
        return P, L, U
    else:
        return U

# ref((ref((A.select_cols(0, 1, 2, 3).row_join(B)), verbosity=2, matrices=1)).subs({a: 2, b: -3}))

In [12]:
def is_zero(expr) -> bool:
    # returns True if expr can be 0 for real values of unknowns
    # use is_complex rather than is_real as symbol.is_complex and 
    # symbol.is_real returns None

    def is_real_dict(case) -> bool:
        for _, value in case.items():
            # zero is considered complex
            if value.is_complex and value != 0:
                return False
        return True

    if (not isinstance(expr, sym.Expr)) or isinstance(expr, sym.Number):
        return expr == 0
    
    # set symbols assumption to true
    real_symbols = sym.symbols(f'x:{len(expr.free_symbols)}')
    rev_symbol_dict = defaultdict()
    for symbol, real_symbol in zip(expr.free_symbols, real_symbols):
        rev_symbol_dict[real_symbol] = symbol
        expr = expr.subs({symbol: real_symbol})

    sol = sym.solve(sym.Eq(expr, 0), expr.free_symbols)
    if isinstance(sol, dict):
        sol = [sol]
    for val in sol:
        display(val.subs(rev_symbol_dict))
    
    return len(sol) > 0
    

def find_all_cases(self) -> List:
    cases = []
    det = (self.T @ self).det()
    if len(det.free_symbols) == 0:
        # determinant of the matrix is fixed
        return []
    elif len(det.free_symbols) == 1:
        for sol in sym.solve(det):
            cases.extend([{det.free_symbols.pop(): sol}])
    else:
        for sol in sym.solve(det):
            cases.extend([sol])
    cases = [dict(case) for case in set(tuple(case.items()) for case in cases)]

    # if variable is not in dictionary, it can be treated as entire real
    # except for specific cases found in other combinations
    combinations = set()
    for subset in powerset(cases):
        combined = dict()
        for d in subset:
            combined = {**combined, **d}
        combinations.add(tuple(sym.ordered(combined.items())))
    
    return list(sym.ordered(dict(combination) for combination in combinations))

def evaluate_cases(self, rhs) -> None:
    cases = find_all_cases(self.row_join(rhs))
    all_possible_values = set(possible_val for case in cases for possible_val in case.items())
    return all_possible_values
    for i, case in enumerate(cases, 1):
        exclusions = [dict([val]) for val in all_possible_values.symmetric_difference(set(case.items()))]
        print(f"Case {i}: {case}, not including {exclusions}")
        U = ref(self.row_join(rhs).subs(case), verbosity = 2, matrices = 1, follow_GE = False)
        display(U)


evaluate_cases(A, B)
# tmp = find_all_cases(A)

set()

In [7]:
X = Matrix.create_unk_matrix(4, 5)
X

⎡x_1,1  x_1,2  x_1,3  x_1,4  x_1,5⎤
⎢                                 ⎥
⎢x_2,1  x_2,2  x_2,3  x_2,4  x_2,5⎥
⎢                                 ⎥
⎢x_3,1  x_3,2  x_3,3  x_3,4  x_3,5⎥
⎢                                 ⎥
⎣x_4,1  x_4,2  x_4,3  x_4,4  x_4,5⎦

In [31]:
find_all_cases(A)

[{}, {a: 0}, {a: b}]

In [53]:
REF = A.row_join(B).echelon_form()
REF[2, 2]

     3      2  
- 2⋅a  + 2⋅a ⋅b

In [56]:
sym.solve(REF[2, 2])

[{a: 0}, {b: a}]

In [58]:
tmp.echelon_form()

⎡0  b + 2   3        3     ⎤
⎢                          ⎥
⎢0    0    -3⋅b   2 - 2⋅b  ⎥
⎢                          ⎥
⎢                 2        ⎥
⎣0    0     0    b  + b - 2⎦

In [51]:
tmp = A.row_join(B).subs({a: 0, b: b})

sym.solve(sym.Mul(*tmp.echelon_form().get_pivot_elements()))

⎡                ⎧    2⋅b ⎫⎤
⎢{a: 0}, {a: b}, ⎨a: ─────⎬⎥
⎣                ⎩   b + 2⎭⎦

In [36]:
A.row_join(B).subs({a: b}).echelon_form()

⎡2⋅b  b + 2   3       3     ⎤
⎢                           ⎥
⎢        3     2     2      ⎥
⎢ 0    -b    -b   - b  + 2⋅b⎥
⎢                           ⎥
⎢                       3   ⎥
⎣ 0     0     0     -2⋅b    ⎦

In [14]:
C = A.row_join(B)
find_all_cases(C)

[]

In [80]:
A.adj()

TypeError: super() argument 1 must be a type, not Matrix

In [20]:
a, b, c = sym.symbols('a b c')
evaluate_cases(Matrix([
    [a,1,0,0,0],
    [1,b,1,0,0],
    [0,1,c,1,0],
    [0,0,1,b,1],
    [0,0,0,1,a]
]), rhs=Matrix([0, 0, 0, 0, 0]))

Case 1: {}, not including [{a: c/(b*c - 2)}, {a: 1/b}]
R_1 <-> R_2


⎡1  b  1  0  0  0⎤
⎢                ⎥
⎢a  1  0  0  0  0⎥
⎢                ⎥
⎢0  1  c  1  0  0⎥
⎢                ⎥
⎢0  0  1  b  1  0⎥
⎢                ⎥
⎣0  0  0  1  a  0⎦



R_2 <- R_2 - (a)R_1


⎡1     b      1   0  0  0⎤
⎢                        ⎥
⎢0  -a⋅b + 1  -a  0  0  0⎥
⎢                        ⎥
⎢0     1      c   1  0  0⎥
⎢                        ⎥
⎢0     0      1   b  1  0⎥
⎢                        ⎥
⎣0     0      0   1  a  0⎦



R_2 <-> R_3


...



R_5 <- R_5 - (1)R_4


⎡   1     b  1   0             0           0⎤
⎢                                           ⎥
⎢   c                                       ⎥
⎢───────  1  0   0             0           0⎥
⎢b⋅c - 2                                    ⎥
⎢                                           ⎥
⎢   0     0  1   b             1           0⎥
⎢                                           ⎥
⎢                              c            ⎥
⎢   0     0  0   1          ───────        0⎥
⎢                           b⋅c - 2         ⎥
⎢                                           ⎥
⎢                           2               ⎥
⎢  -c                    b⋅c         c      ⎥
⎢───────  0  0  -b⋅c  - ─────── + ───────  0⎥
⎣b⋅c - 2                b⋅c - 2   b⋅c - 2   ⎦



R_5 <- R_5 - (-b*c)R_4


⎡   1     b  1  0     0     0⎤
⎢                            ⎥
⎢   c                        ⎥
⎢───────  1  0  0     0     0⎥
⎢b⋅c - 2                     ⎥
⎢                            ⎥
⎢   0     0  1  b     1     0⎥
⎢                            ⎥
⎢                     c      ⎥
⎢   0     0  0  1  ───────  0⎥
⎢                  b⋅c - 2   ⎥
⎢                            ⎥
⎢  -c                 c      ⎥
⎢───────  0  0  0  ───────  0⎥
⎣b⋅c - 2           b⋅c - 2   ⎦

⎡   1     b  1  0     0     0⎤
⎢                            ⎥
⎢   c                        ⎥
⎢───────  1  0  0     0     0⎥
⎢b⋅c - 2                     ⎥
⎢                            ⎥
⎢   0     0  1  b     1     0⎥
⎢                            ⎥
⎢                     c      ⎥
⎢   0     0  0  1  ───────  0⎥
⎢                  b⋅c - 2   ⎥
⎢                            ⎥
⎢  -c                 c      ⎥
⎢───────  0  0  0  ───────  0⎥
⎣b⋅c - 2           b⋅c - 2   ⎦

In [77]:
A

⎡2⋅a  b + 2  3⎤
⎢             ⎥
⎢ 2           ⎥
⎢a      b    a⎥
⎢             ⎥
⎣ a   b + 1  2⎦

# REF

In [8]:
a, b = sym.symbols('a b')

A = Matrix([[a,2,a,a+b],
            [a,2,a,a],
            [3,3,-b,3],
            [a+1,3,a+1,a+1]])

B = Matrix([a-b, a-b, -b, a-b+1])
display(A, B)

A.row_join(B).ref(verbosity=0)

⎡  a    2    a    a + b⎤
⎢                      ⎥
⎢  a    2    a      a  ⎥
⎢                      ⎥
⎢  3    3   -b      3  ⎥
⎢                      ⎥
⎣a + 1  3  a + 1  a + 1⎦

⎡  a - b  ⎤
⎢         ⎥
⎢  a - b  ⎥
⎢         ⎥
⎢   -b    ⎥
⎢         ⎥
⎣a - b + 1⎦

Exception encountered: name 'is_zero' is not defined


⎛⎡0  0  1  0⎤  ⎡  3    3   -b      3       -b    ⎤⎞
⎜⎢          ⎥  ⎢                                 ⎥⎟
⎜⎢0  1  0  0⎥  ⎢  a    2    a      a      a - b  ⎥⎟
⎜⎢          ⎥, ⎢                                 ⎥⎟
⎜⎢1  0  0  0⎥  ⎢  a    2    a    a + b    a - b  ⎥⎟
⎜⎢          ⎥  ⎢                                 ⎥⎟
⎝⎣0  0  0  1⎦  ⎣a + 1  3  a + 1  a + 1  a - b + 1⎦⎠

In [12]:
a, b = sym.symbols('a b')

A = Matrix([[1, a, b, a],
            [1, 2, b, a],
            [1, a, 0, -(b-2)],
            [1, 2, b-a, 0]])

B = Matrix([1, 1, 1, 0])

display(A, B)

A.row_join(B).ref()

⎡1  a    b       a  ⎤
⎢                   ⎥
⎢1  2    b       a  ⎥
⎢                   ⎥
⎢1  a    0     2 - b⎥
⎢                   ⎥
⎣1  2  -a + b    0  ⎦

⎡1⎤
⎢ ⎥
⎢1⎥
⎢ ⎥
⎢1⎥
⎢ ⎥
⎣0⎦

Exception encountered: name 'is_zero' is not defined


⎛⎡1  0  0  0⎤  ⎡1  a    b       a    1⎤⎞
⎜⎢          ⎥  ⎢                      ⎥⎟
⎜⎢0  1  0  0⎥  ⎢1  2    b       a    1⎥⎟
⎜⎢          ⎥, ⎢                      ⎥⎟
⎜⎢0  0  1  0⎥  ⎢1  a    0     2 - b  1⎥⎟
⎜⎢          ⎥  ⎢                      ⎥⎟
⎝⎣0  0  0  1⎦  ⎣1  2  -a + b    0    0⎦⎠

In [2]:
a, b, = sym.symbols('a b', is_real=True)
A = Matrix([[2*a, b+2, 3],
            [a**2, b, a],
            [a, b+1, 2]])
B = Matrix([3, a+1, 2])

display(A, B)

_, my_ref = A.row_join(B).ref()
my_ref

⎡2⋅a  b + 2  3⎤
⎢             ⎥
⎢ 2           ⎥
⎢a      b    a⎥
⎢             ⎥
⎣ a   b + 1  2⎦

⎡  3  ⎤
⎢     ⎥
⎢a + 1⎥
⎢     ⎥
⎣  2  ⎦

R_2 <- R_2 - (a/2)R_1


⎡2⋅a      b + 2       3     3  ⎤
⎢                              ⎥
⎢       a⋅b          -a       a⎥
⎢ 0   - ─── - a + b  ───  1 - ─⎥
⎢        2            2       2⎥
⎢                              ⎥
⎣ a       b + 1       2     2  ⎦



R_3 <- R_3 - (1/2)R_1


⎡2⋅a      b + 2       3     3  ⎤
⎢                              ⎥
⎢       a⋅b          -a       a⎥
⎢ 0   - ─── - a + b  ───  1 - ─⎥
⎢        2            2       2⎥
⎢                              ⎥
⎢           b                  ⎥
⎢ 0         ─        1/2   1/2 ⎥
⎣           2                  ⎦



ERROR: Max tries exceeded to reduce row 3 with row 2


⎡2⋅a      b + 2       3     3  ⎤
⎢                              ⎥
⎢       a⋅b          -a       a⎥
⎢ 0   - ─── - a + b  ───  1 - ─⎥
⎢        2            2       2⎥
⎢                              ⎥
⎢           b                  ⎥
⎢ 0         ─        1/2   1/2 ⎥
⎣           2                  ⎦

In [76]:
evaluate_cases(A, B)

Case 1: {}, not including [{a: 0}, {a: b}]
ERROR: Max tries exceeded to reduce row 3 with row 2


⎡2⋅a      b + 2       3     3  ⎤
⎢                              ⎥
⎢       a⋅b          -a       a⎥
⎢ 0   - ─── - a + b  ───  1 - ─⎥
⎢        2            2       2⎥
⎢                              ⎥
⎢           b                  ⎥
⎢ 0         ─        1/2   1/2 ⎥
⎣           2                  ⎦

Case 2: {a: 0}, not including [{a: b}]
ERROR: Max tries exceeded to reduce row 2 with row 1
ERROR: Max tries exceeded to reduce row 3 with row 1


⎡0  b + 2  3    3  ⎤
⎢                  ⎥
⎢0   -2    -3   -2 ⎥
⎢                  ⎥
⎣0  -1/3   0   -1/3⎦

Case 3: {a: b}, not including [{a: 0}]
ERROR: Max tries exceeded to reduce row 3 with row 2


⎡2⋅b  b + 2   3     3  ⎤
⎢                      ⎥
⎢       2              ⎥
⎢     -b     -b       b⎥
⎢ 0   ────   ───  1 - ─⎥
⎢      2      2       2⎥
⎢                      ⎥
⎢       b              ⎥
⎢ 0     ─    1/2   1/2 ⎥
⎣       2              ⎦

In [9]:
my_ref

⎡2⋅a      b + 2       3     3  ⎤
⎢                              ⎥
⎢       a⋅b          -a       a⎥
⎢ 0   - ─── - a + b  ───  1 - ─⎥
⎢        2            2       2⎥
⎢                              ⎥
⎢           b                  ⎥
⎢ 0         ─        1/2   1/2 ⎥
⎣           2                  ⎦

In [10]:
my_ref.copy().swap_row(1, 2).ref()

R_3 <- R_3 - (2)R_2


⎡2⋅a    b + 2       3      3 ⎤
⎢                            ⎥
⎢         b                  ⎥
⎢ 0       ─        1/2    1/2⎥
⎢         2                  ⎥
⎢                            ⎥
⎢       a⋅b        a      -a ⎥
⎢ 0   - ─── - a  - ─ - 1  ───⎥
⎣        2         2       2 ⎦



R_3 <- R_3 - (-a)R_2


⎡2⋅a  b + 2   3    3 ⎤
⎢                    ⎥
⎢       b            ⎥
⎢ 0     ─    1/2  1/2⎥
⎢       2            ⎥
⎢                    ⎥
⎣ 0    -a    -1    0 ⎦



ERROR: Max tries exceeded to reduce row 3 with row 2


⎛               ⎡2⋅a  b + 2   3    3 ⎤⎞
⎜⎡1    0    0⎤  ⎢                    ⎥⎟
⎜⎢           ⎥  ⎢       b            ⎥⎟
⎜⎢0    1    0⎥, ⎢ 0     ─    1/2  1/2⎥⎟
⎜⎢           ⎥  ⎢       2            ⎥⎟
⎜⎣0  2 - a  1⎦  ⎢                    ⎥⎟
⎝               ⎣ 0    -a    -1    0 ⎦⎠

In [69]:
A.row_join(B).subs({a:0})

⎡0  b + 2  3  3⎤
⎢              ⎥
⎢0    b    0  1⎥
⎢              ⎥
⎣0  b + 1  2  2⎦

In [73]:
A.row_join(B).subs({a:0}).copy().swap_row(0, 1).ref()

R_2 <- R_2 - (1)R_1
⎡0    b    0  1⎤
⎢              ⎥
⎢0    2    3  2⎥
⎢              ⎥
⎣0  b + 1  2  2⎦


ERROR: Max tries exceeded to reduce row 2 with row 1
R_3 <- R_3 - (1)R_1
⎡0  b  0  1⎤
⎢          ⎥
⎢0  2  3  2⎥
⎢          ⎥
⎣0  1  2  1⎦


ERROR: Max tries exceeded to reduce row 3 with row 1
R_3 <- R_3 - (2/3)R_2
⎡0   b    0   1  ⎤
⎢                ⎥
⎢0   2    3   2  ⎥
⎢                ⎥
⎣0  -1/3  0  -1/3⎦




⎛⎡1   0   0⎤  ⎡0   b    0   1  ⎤⎞
⎜⎢         ⎥  ⎢                ⎥⎟
⎜⎢1   1   0⎥, ⎢0   2    3   2  ⎥⎟
⎜⎢         ⎥  ⎢                ⎥⎟
⎝⎣1  2/3  1⎦  ⎣0  -1/3  0  -1/3⎦⎠

In [ ]:
a, b = sym.symbols('a b')

A = Matrix([[1, 1, 3, 0],
            [0, 2, 4, 1],
            [2, 0, 4, a],
            [1, a, 2, b]])

B = Matrix([-1, b, a, 0])


# SVD Test


In [2]:
# A = Matrix.create_rand_matrix(3, 6)

# A.fast_svd(option='sym')
A = Matrix([[1, -1],
            [-2, 2],
            [2, -2]])

# A = Matrix([
#   [3, 2, 2],
#   [2, 3, -2]
# ])

# A = Matrix([[3, 1, 1],
#             [-1, 3, 1]])

# A = Matrix([[1, -1, 3],
#             [3, 1, 1]])

# A = Matrix([[1, 0],
#             [0, 1],
#             [1, 1]])

# A = Matrix([[3, 2],
#             [2, 3],
#             [2, -2]])


# A = Matrix([[-18, 13, -4, 4],
#             [2, 19, -4, 12],
#             [-14, 11, -12, 8],
#             [-2, 21, 4, 8]])


# U, S, V = A.fast_svd(option='sym')
U, S, V = A.singular_value_decomposition(1)
assert A == U @ S @ V.T

# U, S, V
# U.applyfunc(lambda x: -x)
# U
# A.singular_value_decomposition()
# (A.T @ A).eigenvects()

A^T A


⎡9   -9⎤
⎢      ⎥
⎣-9  9 ⎦

Characteristic Polynomial


x⋅(x - 18)

Original: (18)I - A


...

([1  -2  2], (0,))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

U, S, V


⎡⎡      2⋅√5  -2⋅√5 ⎤                       ⎤
⎢⎢-1/3  ────  ──────⎥                       ⎥
⎢⎢       5      15  ⎥             ⎡-√2   √2⎤⎥
⎢⎢                  ⎥  ⎡3⋅√2  0⎤  ⎢────  ──⎥⎥
⎢⎢       √5    4⋅√5 ⎥  ⎢       ⎥  ⎢ 2    2 ⎥⎥
⎢⎢2/3    ──    ──── ⎥, ⎢ 0    0⎥, ⎢        ⎥⎥
⎢⎢       5      15  ⎥  ⎢       ⎥  ⎢ √2   √2⎥⎥
⎢⎢                  ⎥  ⎣ 0    0⎦  ⎢ ──   ──⎥⎥
⎢⎢              √5  ⎥             ⎣ 2    2 ⎦⎥
⎢⎢-2/3   0      ──  ⎥                       ⎥
⎣⎣              3   ⎦                       ⎦

In [10]:
(A.T @ A).orthogonally_diagonalize()

Characteristic Polynomial


x⋅(x - 1600)⋅(x - 400)⋅(x - 100)

Original: (1600)I - A


⎡1072  392   -224  176 ⎤
⎢                      ⎥
⎢392   508   176   -536⎥
⎢                      ⎥
⎢-224  176   1408  128 ⎥
⎢                      ⎥
⎣176   -536  128   1312⎦


After RREF:


...


After RREF:


⎛⎡1  0  0  1/4 ⎤           ⎞
⎜⎢             ⎥           ⎟
⎜⎢0  1  0  1/2 ⎥           ⎟
⎜⎢             ⎥, (0, 1, 2)⎟
⎜⎢0  0  1  -1/2⎥           ⎟
⎜⎢             ⎥           ⎟
⎝⎣0  0  0   0  ⎦           ⎠


Eigenvectors:


⎡⎡-1/4⎤⎤
⎢⎢    ⎥⎥
⎢⎢-1/2⎥⎥
⎢⎢    ⎥⎥
⎢⎢1/2 ⎥⎥
⎢⎢    ⎥⎥
⎣⎣ 1  ⎦⎦

⎛⎡-1/5  2/5   4/5  -2/5⎤  ⎡0   0    0    0  ⎤⎞
⎜⎢                     ⎥  ⎢                 ⎥⎟
⎜⎢-2/5  -1/5  2/5  4/5 ⎥  ⎢0  100   0    0  ⎥⎟
⎜⎢                     ⎥, ⎢                 ⎥⎟
⎜⎢2/5   -4/5  2/5  -1/5⎥  ⎢0   0   400   0  ⎥⎟
⎜⎢                     ⎥  ⎢                 ⎥⎟
⎝⎣4/5   2/5   1/5  2/5 ⎦  ⎣0   0    0   1600⎦⎠

In [6]:
U, S, V

⎛                       ⎡√2   -√2        ⎤⎞
⎜                       ⎢──   ────   2/3 ⎥⎟
⎜⎡√2  -√2 ⎤             ⎢2     6         ⎥⎟
⎜⎢──  ────⎥             ⎢                ⎥⎟
⎜⎢2    2  ⎥  ⎡5  0  0⎤  ⎢√2    √2        ⎥⎟
⎜⎢        ⎥, ⎢       ⎥, ⎢──    ──    -2/3⎥⎟
⎜⎢√2   √2 ⎥  ⎣0  3  0⎦  ⎢2     6         ⎥⎟
⎜⎢──   ── ⎥             ⎢                ⎥⎟
⎜⎣2    2  ⎦             ⎢    -2⋅√2       ⎥⎟
⎜                       ⎢0   ──────  -1/3⎥⎟
⎝                       ⎣      3         ⎦⎠

# is_zero

In [107]:
def is_zero(expr) -> bool:
    # returns True if expr can be 0 for real values of unknowns
    # use is_complex rather than is_real as symbol.is_complex and 
    # symbol.is_real returns None

    def is_real_dict(case) -> bool:
        for _, value in case.items():
            # zero is considered complex
            if value.is_complex and value != 0:
                return False
        return True

    if (not isinstance(expr, sym.Expr)) or isinstance(expr, sym.Number):
        return expr == 0
    
    # set symbols assumption to true
    real_symbols = sym.symbols(f'x:{len(expr.free_symbols)}')
    for symbol, real_symbol in zip(expr.free_symbols, real_symbols):
        expr = expr.subs({symbol: real_symbol})

    sol = sym.solve(sym.Eq(expr, 0), expr.free_symbols)
    return len(sol) > 0
    # display(f"{sol=}")
    # for case in sol:
    #     if isinstance(case, dict):
    #         if is_real_dict(case):
    #             return True
    #         else:
    #             continue
    #     elif case.is_symbol or case.is_real:
    #         return True
    #     elif not case.is_complex:
    #         return True
    # return False

# a, b = sym.symbols('a b', real=True)
assert is_zero(-a*b/2-a+b) == True
assert is_zero(a-b) == True
assert is_zero(0) == True
assert is_zero(a*b) == True
assert is_zero(b) == True
assert is_zero(1) == False

n, d = sym.fraction(1)
assert is_zero(d) == False

# is_zero(a*b)

In [106]:
def create_unk_matrix(num_rows: int = 1,
                        num_cols: int = 1,
                        symbol: str = 'x',
                        is_real: bool = True) -> sym.Matrix:
    # Creates a matrix of size rows * cols with entries symbol_ij
    entries = sym.symbols(f'{symbol}_(1:{num_rows+1})\\,(1:{num_cols+1})', is_real=is_real)
    return Matrix(entries).reshape(num_rows, num_cols)

create_unk_matrix(3, 4)

⎡x_1,1  x_1,2  x_1,3  x_1,4⎤
⎢                          ⎥
⎢x_2,1  x_2,2  x_2,3  x_2,4⎥
⎢                          ⎥
⎣x_3,1  x_3,2  x_3,3  x_3,4⎦